In [1]:
from google.colab import drive
import nbformat, os

drive.mount('/content/drive', force_remount=True)
SRC = '/content/drive/MyDrive/Colab Notebooks/Untitled14.ipynb'
DST = '/content/drive/MyDrive/colab_saves/baseline_model.ipynb'

with open(SRC, 'r', encoding='utf-8') as f:
    nb = nbformat.read(f, as_version=4)


nb.metadata.pop('widgets', None)
for cell in nb.cells:
    cell.metadata.pop('widgets', None)
os.makedirs(os.path.dirname(DST), exist_ok=True)
with open(DST, 'w', encoding='utf-8') as f:
    nbformat.write(nb, f)

print("✅ Clean copy saved to", DST)

ValueError: mount failed

In [1]:

import pickle, pandas as pd
from google.colab import drive
import os

drive.mount('/content/drive')
with open('/content/drive/My Drive/colab_saves/company_dfs.pkl', 'rb') as f:
    labeled_groups = pickle.load(f)

labeled_group2 = labeled_groups['group2']
labeled_group3 = labeled_groups['group3']
labeled_group4 = labeled_groups['group4']

for name, group in [('group2', labeled_group2),
                    ('group3', labeled_group3),
                    ('group4', labeled_group4)]:
    print(f"\n{name}:")
    for sym, df in group.items():
        print(sym, df.shape, list(df.columns)[:5])



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

group2:
C (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCount']
GS (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCount']
MS (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCount']
USB (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCount']

group3:
AAPL (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCount']
MSFT (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCount']
AMZN (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCount']
GOOGL (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.commentCount']

group4:
META (400, 21) ['id', 'type', 'publishOn', 'attributes.isLockedPro', 'attributes.co

In [2]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, recall_score
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
qwen25_model_id = "Qwen/Qwen2.5-1.5B-Instruct"
qwen25_tok = AutoTokenizer.from_pretrained(qwen25_model_id, use_fast=True)
qwen25_mdl = AutoModelForCausalLM.from_pretrained(
    qwen25_model_id,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",)
qwen25_mdl.eval()




/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

In [3]:
from math import ceil

def llm_binary_predict_batched(
    titles,
    tok,
    mdl,
    batch_size=32,
    max_new_tokens=1,
    trunc_len=128,
    heartbeat_every=50,):


    preds = []
    mdl.eval()

    prompt_tpl = 'Headline: "{title}"\nAnswer strictly 1 if price will go up, else 0.'

    n = len(titles)
    num_batches = ceil(n / batch_size)

    for bi in range(num_batches):
        start = bi * batch_size
        end = min((bi + 1) * batch_size, n)
        batch_titles = titles[start:end]

        prompts = [prompt_tpl.format(title=t) for t in batch_titles]

        try:

            enc = tok(
                prompts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=trunc_len,
            ).to(mdl.device)

            with torch.no_grad():

                out = mdl.generate(
                    **enc,
                    max_new_tokens=max_new_tokens,
                    do_sample=False,
                    num_beams=1,
                    use_cache=True,)


            gen_only = out[:, enc["input_ids"].shape[1]:]
            texts = tok.batch_decode(gen_only, skip_special_tokens=True)

            for txt in texts:
                s = txt.strip()
                s_clean = s.lower().strip()
                if s_clean.startswith("1"):
                    preds.append(1)
                elif s_clean.startswith("0"):
                    preds.append(0)
                else:
                    preds.append(1 if "1" in s_clean else 0)

        except Exception as e:
            print(f"[warn] batch {bi+1}/{num_batches} failed: {e}")
            preds.extend([0] * (end - start))

        if (bi + 1) % heartbeat_every == 0:
            print(f"  processed {end}/{n} titles...", flush=True)
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

    return preds[:n]


def evaluate_group(group_dict, tok, mdl, sample_n=None, seed=42):

    df_all = pd.concat(group_dict.values(), ignore_index=True)

    if sample_n is not None and sample_n < len(df_all):
        df_all = df_all.sample(sample_n, random_state=seed).reset_index(drop=True)

    titles = df_all["title"].astype(str).tolist()
    labels = df_all["label"].astype(int).tolist()

    preds = llm_binary_predict_batched(
        titles,
        tok,
        mdl,
        batch_size=32,
        max_new_tokens=1,
        trunc_len=128,
        heartbeat_every=50,)

    acc = accuracy_score(labels, preds)
    micro_f1 = f1_score(labels, preds, average="micro")
    macro_f1 = f1_score(labels, preds, average="macro")
    weighted_f1 = f1_score(labels, preds, average="weighted")
    macro_recall = recall_score(labels, preds, average="macro")

    return {
        "Accuracy": acc,
        "Micro-F1": micro_f1,
        "Macro-F1": macro_f1,
        "Weighted-F1": weighted_f1,
        "Macro-Recall": macro_recall}


In [4]:
results = {}
for group_name, labeled_group in [
    ("group2", labeled_group2),
    ("group3", labeled_group3),
    ("group4", labeled_group4),
]:
    print(f"\n=== Evaluating {group_name} with Qwen2.5-1.5B-Instruct ===")
    metrics = evaluate_group(labeled_group, qwen25_tok, qwen25_mdl, sample_n=100)  # ← set None for full run
    for k, v in metrics.items():
        print(f"{k:12}: {v*100:6.2f}%")
    results[(group_name, "Qwen2.5-1.5B-Instruct")] = metrics


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



=== Evaluating group2 with Qwen2.5-1.5B-Instruct ===


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Accuracy    :  46.00%
Micro-F1    :  46.00%
Macro-F1    :  31.51%
Weighted-F1 :  28.99%
Macro-Recall:  50.00%

=== Evaluating group3 with Qwen2.5-1.5B-Instruct ===


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Accuracy    :  44.00%
Micro-F1    :  44.00%
Macro-F1    :  30.56%
Weighted-F1 :  26.89%
Macro-Recall:  50.00%

=== Evaluating group4 with Qwen2.5-1.5B-Instruct ===


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Accuracy    :  51.00%
Micro-F1    :  51.00%
Macro-F1    :  33.77%
Weighted-F1 :  34.45%
Macro-Recall:  50.00%
